<a href="https://colab.research.google.com/github/hrstbangera/NLP-with-Python/blob/master/finetune_falcon_7b_sharded_freeGPU_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fine-tune Falcon-7b-instruct-sharded model** on a mental health conversational dataset curated by heliosbrahma can be found on Hugging Face.
Links to both the model and dataset are in the notebook.


##Installs and imports

In [1]:
#all installs
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb
!pip install huggingface_hub

#all imports
import torch
import time
from huggingface_hub import notebook_login
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from transformers import TrainingArguments
from trl import SFTTrainer

#ignore warnings
import warnings
warnings.filterwarnings("ignore")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196

##Notebook connection to Hugging face

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
notebook_login()

##Loading the dataset from hugging face

In [4]:
dataset_name = "heliosbrahma/mental_health_chatbot_dataset"
data = load_dataset(dataset_name)
data

Generating train split:   0%|          | 0/172 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 172
    })
})

##Loading the model and Setting up bitsandbytes config

We will use sharded version of falcon-7b-instruct model



In [27]:
model_name = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

##Loading the tokenizer

In [28]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

##Setting up the LoRA config

In [31]:
model_peft = prepare_model_for_kbit_training(model)

lora_alpha = 32 #16
lora_dropout = 0.05 #0.1
lora_rank = 32 #64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

peft_model = get_peft_model(model_peft, peft_config)

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


##Load the trainer

In [33]:
output_dir = "falcon7binstruct_mentalhealthmodel_jan23"
per_device_train_batch_size = 16 #4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100 #100 #500
warmup_ratio = 0.03
lr_scheduler_type = "cosine" #"constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=True
)

##Passing arguments to the SFTT trainer

In [34]:
max_seq_length = 256

trainer = SFTTrainer(
    model=peft_model,
    train_dataset=data['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

In [35]:
# upcasting the layer norms in torch.bfloat16 for more stable training
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.bfloat16)

##Train the model

You can check your training time if you are doing multiple experiments

In [12]:
start = time.time()

In [13]:
peft_model.config.use_cache = False
trainer.train()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,1.656800
20,1.301300
30,0.912100
40,0.459900
50,0.170700
60,0.066200
70,0.045700
80,0.040500
90,0.038800
100,0.038000


TrainOutput(global_step=100, training_loss=0.4729997545480728, metrics={'train_runtime': 660.6616, 'train_samples_per_second': 9.687, 'train_steps_per_second': 0.151, 'total_flos': 5.302835892765082e+16, 'train_loss': 0.4729997545480728, 'epoch': 36.36})

In [14]:
end=time.time()

In [15]:
time_taken=end-start
print(time_taken)

675.8279531002045


##Save the model

In [16]:
#trainer.save() #if you want to save your model locally

##Push to hub

In [17]:
trainer.push_to_hub()

events.out.tfevents.1706003098.cc1f1b0dfb91.1322.0:   0%|          | 0.00/7.47k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/HarshithNLP/falcon7binstruct_mentalhealthmodel_oct23/commit/1fafaeef660e2e597e492a01fccb1b63da9cf8c6', commit_message='End of training', commit_description='', oid='1fafaeef660e2e597e492a01fccb1b63da9cf8c6', pr_url=None, pr_revision=None, pr_num=None)

##Inference

In [42]:
model_name = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [43]:
# Loading PEFT model
PEFT_MODEL = "HarshithNLP/falcon7binstruct_mentalhealthmodel_oct23"
config = PeftConfig.from_pretrained(PEFT_MODEL)
peft_base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

peft_model = PeftModel.from_pretrained(peft_base_model, PEFT_MODEL)

peft_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
peft_tokenizer.pad_token = peft_tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [44]:
# Generate responses from both orignal model and fine-tuned model
def get_response(question):
  prompt = f"""
  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'

  ###Question: {question}

  ###Response:

  """

  encoding = tokenizer(prompt, return_tensors="pt").to("cuda:0")
  outputs = model.generate(input_ids=encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = tokenizer.eos_token_id, \
                                                                                                                     eos_token_id = tokenizer.eos_token_id, attention_mask = encoding.attention_mask, \
                                                                                                                     temperature=0.1, top_p=0.1, repetition_penalty=1.2, num_return_sequences=1,))
  text_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

  #print(dashline)
  print(f'Response from original falcon_7b_instruct_sharded:\n{text_output}')

  print("*******************************************************")

  peft_encoding = peft_tokenizer(prompt, return_tensors="pt").to("cuda:0")
  peft_outputs = peft_model.generate(input_ids=peft_encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = peft_tokenizer.eos_token_id, \
                                                                                                                      eos_token_id = peft_tokenizer.eos_token_id, attention_mask = peft_encoding.attention_mask, \
                                                                                                                      temperature=0.1, top_p=0.1, repetition_penalty=1.2, num_return_sequences=1,))
  peft_text_output = peft_tokenizer.decode(peft_outputs[0], skip_special_tokens=True)

  print(f'Response from fine-tuned falcon_7b_instruct_sharded:\n{peft_text_output}')


In [45]:
get_response("Are there cures for mental health problems?")

Response from original falcon_7b_instruct_sharded:

  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'

  ###Question: Are there cures for mental health problems?

  ###Response:

  ###Option 1: Yes, there are cures for mental health problems.

  ###Option 2: No, there are no cures for mental health problems.

  ###Option 3: It depends on the individual case.

  ###Option 4: It depends on the individual case.

  ###Option 5: It depends on the individual case.

  ###Option 6: It depends on the individual case.

  ###Option 7: It depends on the individual case.

  ###Option 8: It depends on the individual case.

  ###Option 9: It depends on the individual case.

  ###Option 10: It depends on the individual case.

  ###Option 11: It depends on the individual case.

  ###Option 12: It depends on the individual case.

  ###Option 13: It depends on the indi

In [46]:
get_response("What’s the difference between psychotherapy and counselling?")

Response from original falcon_7b_instruct_sharded:

  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'

  ###Question: What’s the difference between psychotherapy and counselling?

  ###Response:

  ###Therapy is a broad term that can refer to a range of different approaches to mental health treatment. It can include a variety of different techniques, such as cognitive behavioral therapy, psychodynamic therapy, and mindfulness-based therapies. Counseling, on the other hand, is a more specific term that generally refers to a shorter-term, goal-oriented approach that focuses on helping individuals achieve specific goals or improve their relationships. It may involve techniques from both therapy and counseling, depending on the individual's needs.
*******************************************************
Response from fine-tuned falcon_7b_instruct_sharded:

In [47]:
get_response("What happens in a therapy session?")

Response from original falcon_7b_instruct_sharded:

  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'

  ###Question: What happens in a therapy session?

  ###Response:

  - A therapist will ask questions to understand your thoughts and feelings.
  - They may also use techniques to help you process your emotions and develop coping strategies.
  - The goal of therapy is to help you identify and change negative patterns of thinking and behavior.
  - The therapist may also provide homework assignments to practice what you learn in therapy.
  - The frequency and duration of therapy sessions can vary depending on the individual and their needs.
*******************************************************
Response from fine-tuned falcon_7b_instruct_sharded:

  ###Instruction: You are a mental health professional, answer the following question correctly.
  If yo

In [48]:
get_response("Are neurofeedback and biofeedback the same thing?")

Response from original falcon_7b_instruct_sharded:

  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'

  ###Question: Are neurofeedback and biofeedback the same thing?

  ###Response:

  ###Explanation:
  Neurofeedback and biofeedback are similar in that they both involve the use of technology to help individuals learn how to control their body and mind. However, there are some differences between the two. Neurofeedback is typically used to help individuals learn how to control their brain waves, while biofeedback is used to learn how to control other bodily functions, such as heart rate or blood pressure. Additionally, neurofeedback is often used to treat conditions such as anxiety, depression, and ADHD, while biofeedback is often used to treat conditions such as chronic pain and migraines.
*******************************************************
Res

In [49]:
get_response(" What is insomnia disorder?")

Response from original falcon_7b_instruct_sharded:

  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'

  ###Question:  What is insomnia disorder?

  ###Response:

  ###Explanation: Insomnia disorder is a sleep disorder where a person has difficulty falling asleep or staying asleep. It can be caused by a variety of factors, including anxiety, depression, and certain medications. It can also lead to other health problems if left untreated. It is important to seek medical attention if you suspect you have Insomnia disorder.

Incorrect response: 'Insomnia disorder is a sleep disorder where a person has difficulty falling asleep or staying asleep.'

Correct response: 'Insomnia disorder is a sleep disorder where a person has difficulty falling asleep or staying asleep. It can be caused by a variety of factors, including anxiety, depression, and certain med